In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics

from itertools import product

import altair as alt
#from altair.vega import v3
#from IPython.display import HTML
#alt.renderers.enable('notebook')

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')
structures = pd.read_csv('../input/structures.csv')
scalar_coupling_contributions = pd.read_csv('../input/scalar_coupling_contributions.csv')

print('Train dataset shape is -> rows: {} cols:{}'.format(train.shape[0],train.shape[1]))
print('Test dataset shape is  -> rows: {} cols:{}'.format(test.shape[0],test.shape[1]))
print('Sub dataset shape is  -> rows: {} cols:{}'.format(sub.shape[0],sub.shape[1]))
print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0],structures.shape[1]))
print('Scalar_coupling_contributions dataset shape is  -> rows: {} cols:{}'.format(scalar_coupling_contributions.shape[0],
                                                                                   scalar_coupling_contributions.shape[1]))

Train dataset shape is -> rows: 4658147 cols:6
Test dataset shape is  -> rows: 2505542 cols:5
Sub dataset shape is  -> rows: 2505542 cols:2
Structures dataset shape is  -> rows: 2358657 cols:6
Scalar_coupling_contributions dataset shape is  -> rows: 4658147 cols:8


In [13]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807599999999994
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000000000000
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254799999999999
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300000000001
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400000000001


In [14]:
scalar_coupling_contributions.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.022400000000005,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.034700000000001,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.032500000000001,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.031900000000000,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.022199999999998,0.254585,1.25861,0.272013


In [3]:
train = pd.merge(train, scalar_coupling_contributions, how = 'left',
                  left_on  = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'],
                  right_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])

In [63]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807599999999994,83.022400000000005,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000000000000,-11.034700000000001,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254799999999999,-11.032500000000001,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300000000001,-11.031900000000000,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400000000001,83.022199999999998,0.254585,1.25861,0.272013


In [17]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [4]:
pd.concat([train['scalar_coupling_constant'],scalar_coupling_contributions['fc'] ],axis=1)[:10]

,scalar_coupling_constant,fc
0,84.807599999999994,83.022400000000005
1,-11.257000000000000,-11.034700000000001
2,-11.254799999999999,-11.032500000000001
3,-11.254300000000001,-11.031900000000000
4,84.807400000000001,83.022199999999998
5,-11.254099999999999,-11.031700000000001
6,-11.254799999999999,-11.032400000000001
7,84.809299999999993,83.024100000000004
8,-11.254300000000001,-11.031900000000000
9,84.809500000000000,83.024299999999997


In [30]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958
1,dsgdb9nsd_000001,1,H,0.0021504160,-0.0060313176,0.0019761204
2,dsgdb9nsd_000001,2,H,1.0117308430,1.4637511620,0.0002765748
3,dsgdb9nsd_000001,3,H,-0.5408150690,1.4475266140,-0.8766437152
4,dsgdb9nsd_000001,4,H,-0.5238136345,1.4379326440,0.9063972942


In [5]:
from tqdm import tqdm
atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71} # Without fudge factor

fudge_factor = 0.05
atomic_radius = {k:v + fudge_factor for k,v in atomic_radius.items()}
print(atomic_radius)

electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}

#structures = pd.read_csv(structures, dtype={'atom_index':np.int8})

atoms = structures['atom'].values
atoms_en = [electronegativity[x] for x in tqdm(atoms)]
atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]

structures['EN'] = atoms_en
structures['rad'] = atoms_rad

display(structures.head())

 18%|█▊        | 416465/2358657 [00:00<00:01, 1765270.18it/s]

{'H': 0.43, 'C': 0.8200000000000001, 'N': 0.8, 'O': 0.78, 'F': 0.76}


100%|██████████| 2358657/2358657 [00:00<00:00, 3873794.88it/s]


,molecule_name,atom_index,atom,x,y,z,EN,rad
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958,2.55,0.82
1,dsgdb9nsd_000001,1,H,0.0021504160,-0.0060313176,0.0019761204,2.20,0.43
2,dsgdb9nsd_000001,2,H,1.0117308430,1.4637511620,0.0002765748,2.20,0.43
3,dsgdb9nsd_000001,3,H,-0.5408150690,1.4475266140,-0.8766437152,2.20,0.43
4,dsgdb9nsd_000001,4,H,-0.5238136345,1.4379326440,0.9063972942,2.20,0.43


In [6]:
i_atom = structures['atom_index'].values
p = structures[['x', 'y', 'z']].values
p_compare = p
m = structures['molecule_name'].values
m_compare = m
r = structures['rad'].values
r_compare = r

source_row = np.arange(len(structures))
max_atoms = 28

bonds = np.zeros((len(structures)+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((len(structures)+1, max_atoms+1), dtype=np.float32)

print('Calculating bonds')

for i in tqdm(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > len(structures), mask==0), len(structures), target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds')

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in tqdm(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(tqdm(bond_dists))]
bond_lengths_mean = [ np.mean(x) for x in bond_lengths]
bond_lengths_std = [ np.std(x) for x in bond_lengths]
n_bonds = [len(x) for x in bonds_numeric]

#bond_data = {'bond_' + str(i):col for i, col in enumerate(np.transpose(bonds))}
#bond_data.update({'bonds_numeric':bonds_numeric, 'n_bonds':n_bonds})

bond_data = {'n_bonds':n_bonds, 'bond_lengths_mean': bond_lengths_mean,'bond_lengths_std':bond_lengths_std }
bond_df = pd.DataFrame(bond_data)
structures = structures.join(bond_df)
display(structures.head(20))

  0%|          | 0/27 [00:00<?, ?it/s]

Calculating bonds


  1%|          | 18794/2358657 [00:00<00:12, 187936.59it/s]

Counting and condensing bonds


100%|██████████| 2358657/2358657 [00:15<00:00, 156483.14it/s]


,molecule_name,atom_index,atom,x,y,z,EN,rad,n_bonds,bond_lengths_mean,bond_lengths_std
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958,2.55,0.82,4,1.091949701309204,0.000002762467830
1,dsgdb9nsd_000001,1,H,0.0021504160,-0.0060313176,0.0019761204,2.20,0.43,1,1.091953039169312,0.000000000000000
2,dsgdb9nsd_000001,2,H,1.0117308430,1.4637511620,0.0002765748,2.20,0.43,1,1.091951608657837,0.000000000000000
3,dsgdb9nsd_000001,3,H,-0.5408150690,1.4475266140,-0.8766437152,2.20,0.43,1,1.091946363449097,0.000000000000000
4,dsgdb9nsd_000001,4,H,-0.5238136345,1.4379326440,0.9063972942,2.20,0.43,1,1.091947555541992,0.000000000000000
5,dsgdb9nsd_000002,0,N,-0.0404260543,1.0241077530,0.0625637998,3.04,0.80,3,1.017194986343384,0.000009144474461
6,dsgdb9nsd_000002,1,H,0.0172574639,0.0125452063,-0.0273771593,2.20,0.43,1,1.017189979553223,0.000000000000000
7,dsgdb9nsd_000002,2,H,0.9157893661,1.3587451950,-0.0287577581,2.20,0.43,1,1.017187237739563,0.000000000000000
8,dsgdb9nsd_000002,3,H,-0.5202777357,1.3435321260,-0.7755426124,2.20,0.43,1,1.017207860946655,0.000000000000000
9,dsgdb9nsd_000003,0,O,-0.0343604951,0.9775395708,0.0076015923,3.44,0.78,2,0.962106823921204,0.000000000000000


In [7]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    #df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)


In [40]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_index_x,atom_0,x_0,y_0,...,atom_index_y,atom_1,x_1,y_1,z_1,EN_y,rad_y,n_bonds_y,bond_lengths_mean_y,bond_lengths_std_y
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807599999999994,1,H,0.002150416,-0.0060313176,...,0,C,-0.0126981359,1.085804158,0.0080009958,2.55,0.82,4,1.091949701309204,0.00000276246783
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000000000000,1,H,0.002150416,-0.0060313176,...,2,H,1.0117308430,1.463751162,0.0002765748,2.20,0.43,1,1.091951608657837,0.00000000000000
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254799999999999,1,H,0.002150416,-0.0060313176,...,3,H,-0.5408150690,1.447526614,-0.8766437152,2.20,0.43,1,1.091946363449097,0.00000000000000
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300000000001,1,H,0.002150416,-0.0060313176,...,4,H,-0.5238136345,1.437932644,0.9063972942,2.20,0.43,1,1.091947555541992,0.00000000000000
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400000000001,2,H,1.011730843,1.4637511620,...,0,C,-0.0126981359,1.085804158,0.0080009958,2.55,0.82,4,1.091949701309204,0.00000276246783


In [68]:
train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)
train['dist_x'] = (train['x_0'] - train['x_1']) ** 2
test['dist_x'] = (test['x_0'] - test['x_1']) ** 2
train['dist_y'] = (train['y_0'] - train['y_1']) ** 2
test['dist_y'] = (test['y_0'] - test['y_1']) ** 2
train['dist_z'] = (train['z_0'] - train['z_1']) ** 2
test['dist_z'] = (test['z_0'] - test['z_1']) ** 2

train['type_0'] = train['type'].apply(lambda x: x[0])
test['type_0'] = test['type'].apply(lambda x: x[0])

In [69]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [70]:
def create_features(df):
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['dist'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['dist'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['dist'].transform('max')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')
    df[f'molecule_atom_index_0_x_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['x_1'].transform('std')
    df[f'molecule_atom_index_0_y_1_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('mean')
    df[f'molecule_atom_index_0_y_1_mean_diff'] = df[f'molecule_atom_index_0_y_1_mean'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_mean_div'] = df[f'molecule_atom_index_0_y_1_mean'] / df['y_1']
    df[f'molecule_atom_index_0_y_1_max'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('max')
    df[f'molecule_atom_index_0_y_1_max_diff'] = df[f'molecule_atom_index_0_y_1_max'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('std')
    df[f'molecule_atom_index_0_z_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['z_1'].transform('std')
    df[f'molecule_atom_index_0_dist_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('mean')
    df[f'molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['dist']
    df[f'molecule_atom_index_0_dist_mean_div'] = df[f'molecule_atom_index_0_dist_mean'] / df['dist']
    df[f'molecule_atom_index_0_dist_max'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('max')
    df[f'molecule_atom_index_0_dist_max_diff'] = df[f'molecule_atom_index_0_dist_max'] - df['dist']
    df[f'molecule_atom_index_0_dist_max_div'] = df[f'molecule_atom_index_0_dist_max'] / df['dist']
    df[f'molecule_atom_index_0_dist_min'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df[f'molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['dist']
    df[f'molecule_atom_index_0_dist_min_div'] = df[f'molecule_atom_index_0_dist_min'] / df['dist']
    df[f'molecule_atom_index_0_dist_std'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('std')
    df[f'molecule_atom_index_0_dist_std_diff'] = df[f'molecule_atom_index_0_dist_std'] - df['dist']
    df[f'molecule_atom_index_0_dist_std_div'] = df[f'molecule_atom_index_0_dist_std'] / df['dist']
    df[f'molecule_atom_index_1_dist_mean'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('mean')
    df[f'molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['dist']
    df[f'molecule_atom_index_1_dist_mean_div'] = df[f'molecule_atom_index_1_dist_mean'] / df['dist']
    df[f'molecule_atom_index_1_dist_max'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('max')
    df[f'molecule_atom_index_1_dist_max_diff'] = df[f'molecule_atom_index_1_dist_max'] - df['dist']
    df[f'molecule_atom_index_1_dist_max_div'] = df[f'molecule_atom_index_1_dist_max'] / df['dist']
    df[f'molecule_atom_index_1_dist_min'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('min')
    df[f'molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['dist']
    df[f'molecule_atom_index_1_dist_min_div'] = df[f'molecule_atom_index_1_dist_min'] / df['dist']
    df[f'molecule_atom_index_1_dist_std'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('std')
    df[f'molecule_atom_index_1_dist_std_diff'] = df[f'molecule_atom_index_1_dist_std'] - df['dist']
    df[f'molecule_atom_index_1_dist_std_div'] = df[f'molecule_atom_index_1_dist_std'] / df['dist']
    df[f'molecule_atom_1_dist_mean'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('mean')
    df[f'molecule_atom_1_dist_min'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('min')
    df[f'molecule_atom_1_dist_min_diff'] = df[f'molecule_atom_1_dist_min'] - df['dist']
    df[f'molecule_atom_1_dist_min_div'] = df[f'molecule_atom_1_dist_min'] / df['dist']
    df[f'molecule_atom_1_dist_std'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('std')
    df[f'molecule_atom_1_dist_std_diff'] = df[f'molecule_atom_1_dist_std'] - df['dist']
    df[f'molecule_type_0_dist_std'] = df.groupby(['molecule_name', 'type_0'])['dist'].transform('std')
    df[f'molecule_type_0_dist_std_diff'] = df[f'molecule_type_0_dist_std'] - df['dist']
    df[f'molecule_type_dist_mean'] = df.groupby(['molecule_name', 'type'])['dist'].transform('mean')
    df[f'molecule_type_dist_mean_diff'] = df[f'molecule_type_dist_mean'] - df['dist']
    df[f'molecule_type_dist_mean_div'] = df[f'molecule_type_dist_mean'] / df['dist']
    df[f'molecule_type_dist_max'] = df.groupby(['molecule_name', 'type'])['dist'].transform('max')
    df[f'molecule_type_dist_min'] = df.groupby(['molecule_name', 'type'])['dist'].transform('min')
    df[f'molecule_type_dist_std'] = df.groupby(['molecule_name', 'type'])['dist'].transform('std')
    df[f'molecule_type_dist_std_diff'] = df[f'molecule_type_dist_std'] - df['dist']
    df = reduce_mem_usage(df)
    return df

train = create_features(train)
test = create_features(test)

Mem. usage decreased to 950.66 Mb (69.9% reduction)
Mem. usage decreased to 477.89 Mb (70.2% reduction)


In [71]:
del_cols_list = ['id','molecule_name','sd','pso','dso']
def del_cols(df, cols):
    del_cols_list_ = [l for l in del_cols_list if l in df]
    df = df.drop(del_cols_list_,axis=1)
    return df

train = del_cols(train,del_cols_list)
test = del_cols(test,del_cols_list)

In [72]:
def encode_categoric_single(df):
    lbl = LabelEncoder()
    cat_cols=[]
    try:
        cat_cols = df.describe(include=['O']).columns.tolist()
        for cat in cat_cols:
            df[cat] = lbl.fit_transform(list(df[cat].values))
    except Exception as e:
        print('error: ', str(e) )

    return df

In [53]:
def encode_categoric(dtrain,dtest):
    lbl = LabelEncoder()
    objs_n = len(dtrain)
    dfmerge = pd.concat(objs=[dtrain,dtest],axis=0)
    cat_cols=[]
    try:
        cat_cols = dfmerge.describe(include=['O']).columns.tolist()
        for cat in cat_cols:
            dfmerge[cat] = lbl.fit_transform(list(dfmerge[cat].values))
    except Exception as e:
        print('error: ', str(e) )

    dtrain = dfmerge[:objs_n]
    dtest = dfmerge[objs_n:]
    return dtrain,dtest

In [73]:
train = encode_categoric_single(train)
test = encode_categoric_single(test)

In [74]:
y_fc = train['fc']
X = train.drop(['scalar_coupling_constant','fc'],axis=1)
y = train['scalar_coupling_constant']

X_test = test.copy()

In [76]:
print('X size',X.shape)
print('X_test size',X_test.shape)
print('dtest size',test.shape)
print('y_fc size',y_fc.shape)

del train, test
gc.collect()

X size (4658147, 81)
X_test size (2505542, 81)
dtest size (2505542, 81)
y_fc size (4658147,)


147

In [77]:
good_columns = ['type',
 'bond_lengths_mean_y',
 'bond_lengths_std_y',
 'bond_lengths_mean_x',
 'molecule_atom_index_0_dist_min_div',
 'molecule_atom_index_0_dist_std_div',
 'molecule_atom_index_0_dist_mean',
 'molecule_atom_index_0_dist_max',
 'dist_y',
 'molecule_atom_index_1_dist_std_diff',
 'z_0',
 'molecule_type_dist_min',
 'molecule_atom_index_0_y_1_mean_div',
 'dist_x',
 'x_0',
 'y_0',
 'molecule_type_dist_std',
 'molecule_atom_index_0_y_1_std',
 'molecule_dist_mean',
 'molecule_atom_index_0_dist_std_diff',
 'dist_z',
 'molecule_atom_index_0_dist_std',
 'molecule_atom_index_0_x_1_std',
 'molecule_type_dist_std_diff',
 'molecule_type_0_dist_std',
 'dist',
 'molecule_atom_index_0_dist_mean_diff',
 'molecule_atom_index_1_dist_min_div',
 'molecule_atom_index_1_dist_mean_diff',
 'y_1',
 'molecule_type_dist_mean_div',
 'molecule_dist_max',
 'molecule_atom_index_0_dist_mean_div',
 'z_1',
 'molecule_atom_index_0_z_1_std',
 'molecule_atom_index_1_dist_mean_div',
 'molecule_atom_index_1_dist_min_diff',
 'molecule_atom_index_1_dist_mean',
 'molecule_atom_index_1_dist_min',
 'molecule_atom_index_1_dist_max',
 'molecule_type_0_dist_std_diff',
 'molecule_atom_index_0_dist_min_diff',
 'molecule_type_dist_mean_diff',
 'x_1',
 'molecule_atom_index_0_y_1_max',
 'molecule_atom_index_0_y_1_mean_diff',
 'molecule_atom_1_dist_std_diff',
 'molecule_atom_index_0_y_1_mean',
 'molecule_atom_1_dist_std',
 'molecule_type_dist_max']

X = X[good_columns].copy()
X_test = X_test[good_columns].copy()

In [78]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

In [82]:
n_estimators_default = 4000

params = {'num_leaves': 50,
          'min_child_samples': 79,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': 9,
          'learning_rate': 0.2,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0
         }

In [84]:
def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()


def train_model_regression(X, X_test, y, params, folds, model_type='lgb', eval_metric='mae', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000):
    """
    A function to train a variety of regression models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'sklearn_scoring_function': metrics.mean_absolute_error},
                    'group_mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'scoring_function': group_mean_log_mae},
                    'mse': {'lgb_metric_name': 'mse',
                        'catboost_metric_name': 'MSE',
                        'sklearn_scoring_function': metrics.mean_squared_error}
                    }

    
    result_dict = {}
    
    # out-of-fold predictions on train data
    oof = np.zeros(len(X))
    
    # averaged predictions on train data
    prediction = np.zeros(len(X_test))
    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = n_estimators, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
         for Parameter verbosity exceed bound [0,3]
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=metrics_dict[eval_metric]['catboost_metric_name'])
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        if eval_metric != 'group_mae':
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
        else:
            scores.append(metrics_dict[eval_metric]['scoring_function'](y_valid, y_pred_valid, X_valid['type']))

        prediction += y_pred    
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= folds.n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= folds.n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            
            result_dict['feature_importance'] = feature_importance
        
    return result_dict

In [85]:
X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y_fc.values})
X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
result_dict_lgb_oof =  {}
for t in X['type'].unique():
    print(f'Training of type {t}')
    X_t = X.loc[X['type'] == t]
    X_test_t = X_test.loc[X_test['type'] == t]
    y_t = X_short.loc[X_short['type'] == t, 'target']
    result_dict_lgb_oof = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=False,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
    X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb_oof['oof']
    X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb_oof['prediction']

Training of type 0
Fold 1 started at Tue Jul 23 14:03:34 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 1.66158	valid_1's l1: 1.79869
[1000]	training's l1: 1.42358	valid_1's l1: 1.6593
[1500]	training's l1: 1.27245	valid_1's l1: 1.58787
[2000]	training's l1: 1.16067	valid_1's l1: 1.54584
[2500]	training's l1: 1.06967	valid_1's l1: 1.51581
[3000]	training's l1: 0.992123	valid_1's l1: 1.49276
[3500]	training's l1: 0.924613	valid_1's l1: 1.47467
[4000]	training's l1: 0.866075	valid_1's l1: 1.46081
Did not meet early stopping. Best iteration is:
[4000]	training's l1: 0.866075	valid_1's l1: 1.46081
Fold 2 started at Tue Jul 23 14:15:46 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 1.6607	valid_1's l1: 1.79643
[1000]	training's l1: 1.42208	valid_1's l1: 1.65532
[1500]	training's l1: 1.26988	valid_1's l1: 1.58376
[2000]	training's l1: 1.15893	valid_1's l1: 1.54152
[2500]	training's l1: 1.06648	valid_1's l1: 1

[4000]	training's l1: 0.0157084	valid_1's l1: 0.598304
Did not meet early stopping. Best iteration is:
[4000]	training's l1: 0.0157084	valid_1's l1: 0.598304
Fold 4 started at Tue Jul 23 15:10:23 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.334288	valid_1's l1: 0.637713
[1000]	training's l1: 0.188134	valid_1's l1: 0.61469
[1500]	training's l1: 0.115119	valid_1's l1: 0.608283
[2000]	training's l1: 0.0736848	valid_1's l1: 0.604624
[2500]	training's l1: 0.0481025	valid_1's l1: 0.603237
[3000]	training's l1: 0.0322589	valid_1's l1: 0.602693
[3500]	training's l1: 0.0221975	valid_1's l1: 0.602229
[4000]	training's l1: 0.0156317	valid_1's l1: 0.601935
Did not meet early stopping. Best iteration is:
[4000]	training's l1: 0.0156317	valid_1's l1: 0.601935
Fold 5 started at Tue Jul 23 15:12:03 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.333385	valid_1's l1: 0.643799
[1000]	training's l1: 0.18816	valid_1's

[2500]	training's l1: 0.269597	valid_1's l1: 0.406025
[3000]	training's l1: 0.247589	valid_1's l1: 0.399592
[3500]	training's l1: 0.228708	valid_1's l1: 0.394668
[4000]	training's l1: 0.21226	valid_1's l1: 0.39071
Did not meet early stopping. Best iteration is:
[4000]	training's l1: 0.21226	valid_1's l1: 0.39071
Fold 2 started at Tue Jul 23 16:19:05 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.44564	valid_1's l1: 0.490617
[1000]	training's l1: 0.374198	valid_1's l1: 0.450083
[1500]	training's l1: 0.329606	valid_1's l1: 0.429334
[2000]	training's l1: 0.296846	valid_1's l1: 0.417205
[2500]	training's l1: 0.270219	valid_1's l1: 0.408769
[3000]	training's l1: 0.247874	valid_1's l1: 0.402358
[3500]	training's l1: 0.228782	valid_1's l1: 0.397396
[4000]	training's l1: 0.211968	valid_1's l1: 0.39326
Did not meet early stopping. Best iteration is:
[4000]	training's l1: 0.211968	valid_1's l1: 0.39326
Fold 3 started at Tue Jul 23 16:25:06 2019
Trainin

[500]	training's l1: 0.207023	valid_1's l1: 0.28191
[1000]	training's l1: 0.150947	valid_1's l1: 0.264274
[1500]	training's l1: 0.117239	valid_1's l1: 0.256476
[2000]	training's l1: 0.0938037	valid_1's l1: 0.251436
[2500]	training's l1: 0.076519	valid_1's l1: 0.249007
[3000]	training's l1: 0.0632617	valid_1's l1: 0.247397
[3500]	training's l1: 0.0527891	valid_1's l1: 0.245849
[4000]	training's l1: 0.0445088	valid_1's l1: 0.244899
Did not meet early stopping. Best iteration is:
[4000]	training's l1: 0.0445088	valid_1's l1: 0.244899
Fold 5 started at Tue Jul 23 18:37:27 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.207221	valid_1's l1: 0.283178
[1000]	training's l1: 0.151239	valid_1's l1: 0.264408
[1500]	training's l1: 0.117266	valid_1's l1: 0.256296
[2000]	training's l1: 0.0936891	valid_1's l1: 0.251563
[2500]	training's l1: 0.0763814	valid_1's l1: 0.249004
[3000]	training's l1: 0.0632121	valid_1's l1: 0.247048
[3500]	training's l1: 0.0528586

In [86]:
X['oof_fc'] = X_short['oof']
X_test['oof_fc'] = X_short_test['prediction']

In [87]:
good_columns = ['type','oof_fc',
 'bond_lengths_mean_y',
 'bond_lengths_std_y',
 'bond_lengths_mean_x',
 'molecule_atom_index_0_dist_min_div',
 'molecule_atom_index_0_dist_std_div',
 'molecule_atom_index_0_dist_mean',
 'molecule_atom_index_0_dist_max',
 'dist_y',
 'molecule_atom_index_1_dist_std_diff',
 'z_0',
 'molecule_type_dist_min',
 'molecule_atom_index_0_y_1_mean_div',
 'dist_x',
 'x_0',
 'y_0',
 'molecule_type_dist_std',
 'molecule_atom_index_0_y_1_std',
 'molecule_dist_mean',
 'molecule_atom_index_0_dist_std_diff',
 'dist_z',
 'molecule_atom_index_0_dist_std',
 'molecule_atom_index_0_x_1_std',
 'molecule_type_dist_std_diff',
 'molecule_type_0_dist_std',
 'dist',
 'molecule_atom_index_0_dist_mean_diff',
 'molecule_atom_index_1_dist_min_div',
 'molecule_atom_index_1_dist_mean_diff',
 'y_1',
 'molecule_type_dist_mean_div',
 'molecule_dist_max',
 'molecule_atom_index_0_dist_mean_div',
 'z_1',
 'molecule_atom_index_0_z_1_std',
 'molecule_atom_index_1_dist_mean_div',
 'molecule_atom_index_1_dist_min_diff',
 'molecule_atom_index_1_dist_mean',
 'molecule_atom_index_1_dist_min',
 'molecule_atom_index_1_dist_max',
 'molecule_type_0_dist_std_diff',
 'molecule_atom_index_0_dist_min_diff',
 'molecule_type_dist_mean_diff',
 'x_1',
 'molecule_atom_index_0_y_1_max',
 'molecule_atom_index_0_y_1_mean_diff',
 'molecule_atom_1_dist_std_diff',
 'molecule_atom_index_0_y_1_mean',
 'molecule_atom_1_dist_std',
 'molecule_type_dist_max']

X = X[good_columns].copy()
X_test = X_test[good_columns].copy()

In [102]:
params = {'num_leaves': 50,
          'min_child_samples': 79,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': 9,
          'learning_rate': 0.2,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0
         }
n_estimators_default = 4000

In [104]:
X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y.values})
X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
for t in X['type'].unique():
    print(f'Training of type {t}')
    X_t = X.loc[X['type'] == t]
    X_test_t = X_test.loc[X_test['type'] == t]
    y_t = X_short.loc[X_short['type'] == t, 'target']
    result_dict_lgb3 = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='xgb', eval_metric='group_mae', plot_feature_importance=False,
                                                      verbose=1,
                                                      early_stopping_rounds=200, 
                                                      n_estimators=n_estimators_default)
    X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb3['oof']
    X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb3['prediction']

Training of type 0
Fold 1 started at Wed Jul 24 11:22:40 2019


XGBoostError: value -1 for Parameter verbosity exceed bound [0,3]

In [105]:
X_t.head()

,type,oof_fc,bond_lengths_mean_y,bond_lengths_std_y,bond_lengths_mean_x,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_std_div,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_max,dist_y,...,molecule_type_0_dist_std_diff,molecule_atom_index_0_dist_min_diff,molecule_type_dist_mean_diff,x_1,molecule_atom_index_0_y_1_max,molecule_atom_index_0_y_1_mean_diff,molecule_atom_1_dist_std_diff,molecule_atom_index_0_y_1_mean,molecule_atom_1_dist_std,molecule_type_dist_max
0,0,95.471974719504374,1.091796875,0.000002741813660,1.091796875,1.0,0.316406250000,1.6103515625,1.783203125,1.192382812500,...,-1.091796875,0.0,-0.000003397464752,-0.012695312500000,1.4638671875,0.2729492187500,-1.091796875,1.3583984375,0.000003218650818,1.091796875
4,0,92.410355169072020,1.091796875,0.000002741813660,1.091796875,1.0,0.365478515625,1.5527343750,1.783203125,0.142822265625,...,-1.091796875,0.0,-0.000001966953278,-0.012695312500000,1.4472656250,0.2379150390625,-1.091796875,1.3242187500,0.000003218650818,1.091796875
7,0,95.487701961623358,1.091796875,0.000002741813660,1.091796875,1.0,0.447509765625,1.4375000000,1.783203125,0.130859375000,...,-1.091796875,0.0,0.000003278255463,-0.012695312500000,1.4375000000,0.1760253906250,-1.091796875,1.2617187500,0.000003218650818,1.091796875
9,0,96.841014478871855,1.091796875,0.000002741813660,1.091796875,1.0,NaN,1.0917968750,1.091796875,0.124023437500,...,-1.091796875,0.0,0.000002086162567,-0.012695312500000,1.0859375000,0.0000000000000,-1.091796875,1.0859375000,0.000003218650818,1.091796875
17,0,190.909645967823280,1.109375000,0.042572021484375,1.066406250,1.0,0.763671875000,1.6425781250,2.218750000,1.137695312500,...,NaN,0.0,0.000000000000000,-0.013320922851562,1.1328125000,-0.5756835937500,NaN,0.5566406250,NaN,1.066406250


In [96]:
sub['scalar_coupling_constant'] = X_short_test['prediction']
sub.to_csv('submission_type2.csv', index=False)
sub.head()

,id,scalar_coupling_constant
0,4658147,10.206950373000737
1,4658148,188.998549979818335
2,4658149,3.655305082993490
3,4658150,188.924691250002439
4,4658151,8.801439017903467


In [91]:
X_short.head()

,ind,type,oof,target
0,0,0,90.894461165890704,84.8125000
1,1,3,-11.234632799768086,-11.2578125
2,2,3,-10.707915768809444,-11.2578125
3,3,3,-11.831883907076307,-11.2578125
4,4,0,86.448892531602638,84.8125000


In [ ]:
X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y.values})
X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
for t in X['type'].unique():
    print(f'Training of type {t}')
    X_t = X.loc[X['type'] == t]
    X_test_t = X_test.loc[X_test['type'] == t]
    y_t = X_short.loc[X_short['type'] == t, 'target']
    result_dict_lgb3 = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=False,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
    X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb3['oof']
    X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb3['prediction']